# **LSTM**

In [1]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

# Load your data here
data_path = '/kaggle/input/pig-disease-f/data 2/Train'
metadata = pd.read_csv('//kaggle/input/pig-data/data.csv')

class_names = {
    1: 'Pneumonia',
    2: 'PRRS',
    3: 'Swine Fever'
    # Add more class IDs and their corresponding names as needed
}

metadata['class_name'] = metadata['classID'].map(class_names)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Assuming that your audio files are organized in folders named after their classes
class_folders = ['Pnemonia', 'prrs', 'swine_fever']
audio_data = []
labels = []

for folder in class_folders:
    folder_path = os.path.join(data_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            audio, sr = librosa.load(audio_path, sr=None)
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
            audio_data.append(mfccs)
            labels.append(folder)


In [3]:
# Padding MFCCs to ensure uniformity in feature dimensions
max_length = max([mfcc.shape[1] for mfcc in audio_data])
padded_mfccs = [np.pad(mfcc, pad_width=((0, 0), (0, max_length - mfcc.shape[1])), mode='constant') for mfcc in audio_data]

# Converting to NumPy arrays
X = np.array(padded_mfccs)
y = np.array(labels)

# Reshape for LSTM input
X = X.reshape(X.shape[0], X.shape[2], X.shape[1]) # Reshaping to (samples, timesteps, features)

# Label encoding
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [4]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 390, 128)          86528     
                                                                 
 dropout (Dropout)           (None, 390, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 136131 (531.76 KB)
Trainable params: 136131 (531.76 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
num_epochs = 10
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification_lstm.hdf5', verbose=1, save_best_only=True)

start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("Training completed in time: ", datetime.now() - start)

Epoch 1/10
24/26 [==========================>...] - ETA: 0s - loss: 0.7820 - accuracy: 0.7279
Epoch 1: val_loss improved from inf to 0.55989, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 12s 66ms/step - loss: 0.7700 - accuracy: 0.7318 - val_loss: 0.5599 - val_accuracy: 0.8107
Epoch 2/10
 7/26 [=======>......................] - ETA: 0s - loss: 0.6014 - accuracy: 0.7768

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


25/26 [===========================>..] - ETA: 0s - loss: 0.5561 - accuracy: 0.8062
Epoch 2: val_loss improved from 0.55989 to 0.52743, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.5454 - accuracy: 0.8107 - val_loss: 0.5274 - val_accuracy: 0.8058
Epoch 3/10
25/26 [===========================>..] - ETA: 0s - loss: 0.4630 - accuracy: 0.8338
Epoch 3: val_loss improved from 0.52743 to 0.44859, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.4676 - accuracy: 0.8313 - val_loss: 0.4486 - val_accuracy: 0.8204
Epoch 4/10
25/26 [===========================>..] - ETA: 0s - loss: 0.3421 - accuracy: 0.8863
Epoch 4: val_loss improved from 0.44859 to 0.40664, saving model to ./audio_classification_lstm.hdf5
26/26 [==============================] - 1s 28ms/step - loss: 0.3502 - accuracy: 0.8835 - val_loss: 0.4066 - val_accuracy: 0.8544
Epoch 5/10
26/26 [==================

In [6]:
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy[1]}')

Test accuracy: 0.8155339956283569


In [7]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# Predict classes
predict_x = model.predict(X_test) 
predicted_classes = np.argmax(predict_x, axis=1)

# Convert one-hot encoded y_test to label format
true_classes = np.argmax(y_test, axis=1)
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Precision
precision = precision_score(true_classes, predicted_classes, average='weighted')
print(f"Precision: {precision}")

# Recall
recall = recall_score(true_classes, predicted_classes, average='weighted')
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(true_classes, predicted_classes, average='weighted')
print(f"F1 Score: {f1}")

7/7 [==============================] - 1s 14ms/step
Confusion Matrix:
[[135  13   2]
 [ 16  12   2]
 [  5   0  21]]
Precision: 0.8060567587752054
Recall: 0.8155339805825242
F1 Score: 0.8099787134624371


# **ResNet**

In [8]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50
from datetime import datetime

# Load your data here
data_path = '/kaggle/input/pig-disease-f/data 2/Train'
metadata = pd.read_csv('//kaggle/input/pig-data/data.csv')

class_names = {
    1: 'Pneumonia',
    2: 'PRRS',
    3: 'Swine Fever'
}

metadata['class_name'] = metadata['classID'].map(class_names)

In [9]:
# Assuming that your audio files are organized in folders named after their classes
class_folders = ['Pnemonia', 'prrs', 'swine_fever']
audio_data = []
labels = []

for folder in class_folders:
    folder_path = os.path.join(data_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            audio, sr = librosa.load(audio_path, sr=None)
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
            audio_data.append(mfccs)
            labels.append(folder)

In [10]:
class_folders = ['Pnemonia', 'prrs', 'swine_fever']
audio_data = []
labels = []

for folder in class_folders:
    folder_path = os.path.join(data_path, folder)
    for filename in os.listdir(folder_path):
        if filename.endswith('.wav'):
            audio_path = os.path.join(folder_path, filename)
            audio, sr = librosa.load(audio_path, sr=None)
            mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
            audio_data.append(mfccs)
            labels.append(folder)

In [11]:
# Padding MFCCs to ensure uniformity in feature dimensions
max_length = max([mfcc.shape[1] for mfcc in audio_data])
padded_mfccs = [np.pad(mfcc, pad_width=((0, 0), (0, max_length - mfcc.shape[1])), mode='constant') for mfcc in audio_data]

# Converting to NumPy arrays
X = np.array(padded_mfccs)
y = np.array(labels)

# Reshape for ResNet input
# Assuming that the size of the MFCCs is (40, max_length), we need to add a channel dimension
X_reshaped = np.expand_dims(X, axis=-1)

# Label encoding
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size=0.2, random_state=0)


In [12]:
input_shape = X_train.shape[1:]  # Shape of input data

base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 13, 2048)       23581440  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense_1 (Dense)             (None, 256)               524544    
                                                                 
 batch_normalization (Batch  (None, 256)               1024      
 Normalization)                                                  
                                                                 
 dense_2 (Dense)             (None, 3)                 771       
                                                                 
Total params: 24107779 (91.96 MB)
Trainable params: 24

In [13]:
num_epochs = 10
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='./audio_classification_resnet.hdf5', verbose=1, save_best_only=True)

start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
print("Training completed in time: ", datetime.now() - start)

Epoch 1/10
26/26 [==============================] - ETA: 0s - loss: 1.0789 - accuracy: 0.5522
Epoch 1: val_loss improved from inf to 64.96595, saving model to ./audio_classification_resnet.hdf5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - 52s 331ms/step - loss: 1.0789 - accuracy: 0.5522 - val_loss: 64.9660 - val_accuracy: 0.7282
Epoch 2/10
26/26 [==============================] - ETA: 0s - loss: 0.5808 - accuracy: 0.8034
Epoch 2: val_loss improved from 64.96595 to 2.96689, saving model to ./audio_classification_resnet.hdf5
26/26 [==============================] - 5s 176ms/step - loss: 0.5808 - accuracy: 0.8034 - val_loss: 2.9669 - val_accuracy: 0.1942
Epoch 3/10
26/26 [==============================] - ETA: 0s - loss: 0.4009 - accuracy: 0.8507
Epoch 3: val_loss improved from 2.96689 to 1.20774, saving model to ./audio_classification_resnet.hdf5
26/26 [==============================] - 5s 177ms/step - loss: 0.4009 - accuracy: 0.8507 - val_loss: 1.2077 - val_accuracy: 0.4757
Epoch 4/10
26/26 [==============================] - ETA: 0s - loss: 0.2578 - accuracy: 0.9090
Epoch 4: val_loss improved from 1.20774 to 0.96371, saving model to ./audio_classification_resnet.hdf5
26/26 [======

In [14]:
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy[1]}')

Test accuracy: 0.8058252334594727


In [15]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
# Predict classes
predict_x = model.predict(X_test) 
predicted_classes = np.argmax(predict_x, axis=1)

# Convert one-hot encoded y_test to label format
true_classes = np.argmax(y_test, axis=1)
# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Precision
precision = precision_score(true_classes, predicted_classes, average='weighted')
print(f"Precision: {precision}")

# Recall
recall = recall_score(true_classes, predicted_classes, average='weighted')
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(true_classes, predicted_classes, average='weighted')
print(f"F1 Score: {f1}")

7/7 [==============================] - 1s 34ms/step
Confusion Matrix:
[[139   0  11]
 [ 26   1   3]
 [  0   0  26]]
Precision: 0.8410856134157105
Recall: 0.8058252427184466
F1 Score: 0.7514625472658671
